# Capstone Project - The Battle of Neighborhoods

## New store location in Barcelona

<img  src=http://3.bp.blogspot.com/-EsiXnXwH4K8/VSadZk-fiqI/AAAAAAAAE48/z0ch8qF_G1Y/s1600/01%2BHuerto%2Bde%2Bla%2BSagrada%2BFamilia%2BBarcelona.jpg>

## Description of the Problem and a discussion of the background

A provider of agricultural articles and supplies in Spain wants to expand its business line in response to the growing market of urban agriculture in small spaces, which has a special boom in the city of Barcelona, due to the concern for the organic food.

The commercial strategy is to place an own store in a  neighborhoods with an urban farm. It’s important  to be able to expand the business by attracting the attention  of new independent distributors who wish to represent the brand in other cities in Spain and the rest of the European community. For this reason, the neighborhood that has a wide range of ‘Spanish Restaurants’, ‘Hotels’ and ‘Coffee Shops’,  will be prioritized in order to attend business partners from other cities.

## A description of the data and how it will be used to solve the problem

It will be used the data about the location of urban farms in Barcelona from  https://opendata-ajuntament.barcelona.cat/data/es/dataset/horts-urbans.

After cleaning and ordering the data,  the available data on Foursquare will be crossed in order to identify which neigborhoods with an urban farm have a greater offer of ‘Spanish Restaurants’, ‘Hotels’ and ‘Coffee Shops’, which will allow optimizing the decision of where to open a local distributor without spending time  and money visiting the neighborhoods or hiring a real state broker.

In [1]:
# Import libraries
import numpy as np
import json
import pandas as pd

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
# Download the dataset and read it into a pandas dataframe.
# The dataset was downloaded from Open Data Barcelona (BCN) https://opendata-ajuntament.barcelona.cat/en
# Then placed on the GitHub repo for the project.
barcelona = pd.read_csv('https://raw.githubusercontent.com/Javier242/Coursera_Capstone/master/HORTS_URBANS.csv')
barcelona

,CODI_CAPA,CAPA_GENERICA,NOM_CAPA,ED50_COORD_X,ED50_COORD_Y,ETRS89_COORD_X,ETRS89_COORD_Y,LONGITUD,LATITUD,DISTRICTE,BARRI,NOM_DISTRICTE,NOM_BARRI,EQUIPAMENT,ADRECA,TELEFON
0,M009,Medi ambient,Horts urbans,428590.952,4579243.454,428499.036,4579042.548,2.145165,41.359715,3,13,Sants-Montjuïc,la Marina de Port,Hort Urbà Masia Can Mestres,"Camí Can Clos, 1*9",10.0
1,M009,Medi ambient,Horts urbans,432907.109,4589145.239,432815.157,4588944.240,2.195663,41.449268,9,60,Sant Andreu,Sant Andreu,Hort Urbà Parc de la Trinitat,"Pg Santa Coloma, 60",10.0
2,M009,Medi ambient,Horts urbans,427454.616,4586506.529,427362.713,4586305.556,2.130710,41.425028,7,39,Horta-Guinardó,Sant Genís dels Agudells,Hort Urbà Masia Can Soler,"Ctra Sant Cugat, 114*132",10.0
3,M009,Medi ambient,Horts urbans,428121.665,4582340.323,428029.755,4582139.388,2.139188,41.387565,4,19,Les Corts,les Corts,Associació Hort Urbà Espai Colònia Castells - ...,"C Montnegre, 30",NaN
4,M009,Medi ambient,Horts urbans,425821.501,4583114.795,425729.612,4582913.854,2.111585,41.394331,4,21,Les Corts,Pedralbes,Hort Urbà de Pedralbes,"C Castellet, 2",10.0
5,M009,Medi ambient,Horts urbans,433173.050,4589193.170,433081.095,4588992.170,2.198841,41.449722,9,57,Sant Andreu,la Trinitat Vella,Hort Urbà Baró de Viver,"Pg Santa Coloma, 125",10.0
6,M009,Medi ambient,Horts urbans,431603.892,4583984.370,431511.950,4583783.419,2.180647,41.402677,2,6,Eixample,la Sagrada Família,Hort Urbà Sagrada Família,"C Padilla, 199*201",10.0
7,M009,Medi ambient,Horts urbans,433089.246,4585983.306,432997.291,4585782.336,2.198194,41.420805,10,72,Sant Martí,Sant Martí de Provençals,Centre Experimental i Educatiu Agricultura Bio...,"C Menorca, 27",10.0
8,M009,Medi ambient,Horts urbans,427371.748,4581254.418,427279.844,4581053.494,2.130350,41.377718,3,18,Sants-Montjuïc,Sants,Hort Urbà d'en Queni,"C Rosés, 8",NaN
9,M009,Medi ambient,Horts urbans,432274.631,4589561.580,432182.685,4589360.577,2.188045,41.452965,8,53,Nou Barris,la Trinitat Nova,La Casa de l'Aigua-Hort Urbà,"C Garbí, 2*4",10.0


In [3]:
# Rename the necessary columns
barcelona.rename(columns={'NOM_BARRI': 'Neighborhood', 'LONGITUD': 'Longitude', 'LATITUD': 'Latitude', 'NOM_DISTRICTE': 'District'}, inplace=True)
barcelona

,CODI_CAPA,CAPA_GENERICA,NOM_CAPA,ED50_COORD_X,ED50_COORD_Y,ETRS89_COORD_X,ETRS89_COORD_Y,Longitude,Latitude,DISTRICTE,BARRI,District,Neighborhood,EQUIPAMENT,ADRECA,TELEFON
0,M009,Medi ambient,Horts urbans,428590.952,4579243.454,428499.036,4579042.548,2.145165,41.359715,3,13,Sants-Montjuïc,la Marina de Port,Hort Urbà Masia Can Mestres,"Camí Can Clos, 1*9",10.0
1,M009,Medi ambient,Horts urbans,432907.109,4589145.239,432815.157,4588944.240,2.195663,41.449268,9,60,Sant Andreu,Sant Andreu,Hort Urbà Parc de la Trinitat,"Pg Santa Coloma, 60",10.0
2,M009,Medi ambient,Horts urbans,427454.616,4586506.529,427362.713,4586305.556,2.130710,41.425028,7,39,Horta-Guinardó,Sant Genís dels Agudells,Hort Urbà Masia Can Soler,"Ctra Sant Cugat, 114*132",10.0
3,M009,Medi ambient,Horts urbans,428121.665,4582340.323,428029.755,4582139.388,2.139188,41.387565,4,19,Les Corts,les Corts,Associació Hort Urbà Espai Colònia Castells - ...,"C Montnegre, 30",NaN
4,M009,Medi ambient,Horts urbans,425821.501,4583114.795,425729.612,4582913.854,2.111585,41.394331,4,21,Les Corts,Pedralbes,Hort Urbà de Pedralbes,"C Castellet, 2",10.0
5,M009,Medi ambient,Horts urbans,433173.050,4589193.170,433081.095,4588992.170,2.198841,41.449722,9,57,Sant Andreu,la Trinitat Vella,Hort Urbà Baró de Viver,"Pg Santa Coloma, 125",10.0
6,M009,Medi ambient,Horts urbans,431603.892,4583984.370,431511.950,4583783.419,2.180647,41.402677,2,6,Eixample,la Sagrada Família,Hort Urbà Sagrada Família,"C Padilla, 199*201",10.0
7,M009,Medi ambient,Horts urbans,433089.246,4585983.306,432997.291,4585782.336,2.198194,41.420805,10,72,Sant Martí,Sant Martí de Provençals,Centre Experimental i Educatiu Agricultura Bio...,"C Menorca, 27",10.0
8,M009,Medi ambient,Horts urbans,427371.748,4581254.418,427279.844,4581053.494,2.130350,41.377718,3,18,Sants-Montjuïc,Sants,Hort Urbà d'en Queni,"C Rosés, 8",NaN
9,M009,Medi ambient,Horts urbans,432274.631,4589561.580,432182.685,4589360.577,2.188045,41.452965,8,53,Nou Barris,la Trinitat Nova,La Casa de l'Aigua-Hort Urbà,"C Garbí, 2*4",10.0


In [4]:
# Clean up the data removing unnecessary columns.
barcelona.drop(['CODI_CAPA', 'CAPA_GENERICA', 'NOM_CAPA', 'ED50_COORD_X', 'ED50_COORD_Y', 'ETRS89_COORD_X', 'ETRS89_COORD_Y', 'DISTRICTE', 'BARRI', 'EQUIPAMENT', 'ADRECA', 'TELEFON' ], axis=1, inplace=True)
barcelona

,Longitude,Latitude,District,Neighborhood
0,2.145165,41.359715,Sants-Montjuïc,la Marina de Port
1,2.195663,41.449268,Sant Andreu,Sant Andreu
2,2.130710,41.425028,Horta-Guinardó,Sant Genís dels Agudells
3,2.139188,41.387565,Les Corts,les Corts
4,2.111585,41.394331,Les Corts,Pedralbes
5,2.198841,41.449722,Sant Andreu,la Trinitat Vella
6,2.180647,41.402677,Eixample,la Sagrada Família
7,2.198194,41.420805,Sant Martí,Sant Martí de Provençals
8,2.130350,41.377718,Sants-Montjuïc,Sants
9,2.188045,41.452965,Nou Barris,la Trinitat Nova


## Methodology and Exploratory Data Analysis¶

#### The Data Science Workflow for parts 3 & 4 includes:

#### Data Analysis and Location Data:

- Foursquare location data will be leveraged to explore or compare districts around Barcelona.

- Data manipulation and analysis to derive subsets of the initial data.

- Identifying the high traffic areas using data visualisation and tatistical nalysis.

#### Visualization:

- Analysis and plotting visualizations.

- Data visualization using various mapping libraries.

#### Discussion and Conclusions:

- Recomendations and results based on the data analysis.

- Discussion of any limitations and how the results can be used, and any conclusions that can be drawn.